In [348]:
import pandas as pd
import re
#OK so it's simple and not magic but it works
hashtag_regex = re.compile(r'\#(\w*)')


# Sources

NBC News is publishing its database of more than 200,000 tweets that Twitter has tied to "malicious activity" from Russia-linked accounts during the 2016 U.S. presidential election.


https://developers.slashdot.org/story/18/02/17/0038251/nbc-publishes-200000-tweets-tied-to-russian-trolls



The original file has some junk in it which we have to delete before opening with Pandas :

Tweets from confirmed Russian trolls, shows only username, timestamp (in UTC), tweet text, and number of times tweet was retweeted and favorited according to our data",,,,,,,,,,,,,,,,,

From NBC News' story: https://www.nbcnews.com/tech/social-media/now-available-more-200-000-deleted-russian-troll-tweets-n844731,,,,,,,,,,,,,,,,,

"If you publish using the data, please credit NBC News and include a link to this page. Send questions to ben.popken@nbcuni.com or twitter.com/bpopken.",,,,,,,,,,,,,,,,,

Twitter screenname,Date tweet sent,Tweet text,Times retweeted,Times favorited,,,,,,,,,,,,,

ryanmaxwell_1,3/22/2016 18:31,#IslamKills Are you trying to say that there were no terrorist attacks in Europe before refugees were let in?,,,,,,,,,,,,,,,



# Minimal cleanup of the CSV


In [ ]:
fp = file('streamlined_tweets.csv','r')
print ''.join(fp.readlines()[:5])

In [86]:
fp = file('streamlined_tweets.csv','r')
fe = file('fakenews_tweets.csv','w')

with fe as piggy:
    for L in fp.readlines()[3:]:
        piggy.write(L)

df = pd.read_csv('fakenews_tweets.csv')

so the data now looks OK 
there are still a few NaNs in the **Tweet text**
column but this isn't so important.

I decided to rename the columns as it made following examples easier

1. everything to lower case
1. swap out spaces to underscores

In [91]:
' '.join( list( df.columns.values)[:5])

'Twitter screenname Date tweet sent Tweet text Times retweeted Times favorited'

In [268]:
rename_mapper = {}
for x in df.columns.values[:5]:
    rename_mapper[x] = x.lower().replace(' ','_')
rename_mapper

{'Date tweet sent': 'date_tweet_sent',
 'Times favorited': 'times_favorited',
 'Times retweeted': 'times_retweeted',
 'Tweet text': 'tweet_text',
 'Twitter screenname': 'twitter_screenname'}

In [287]:
#Do an inplace as this is a fair chunk of data
df.rename(index=str, 
          columns=rename_mapper,
          copy=False,
          inplace=True)
df.columns.values[:5]

array(['twitter_screenname', 'date_tweet_sent', 'tweet_text',
       'times_retweeted', 'times_favorited'], dtype=object)

so now we can just run a regex on the column 

In [306]:
ph_raw = re.compile(r'\#pokemon\w*', re.IGNORECASE)
# for some reason the fancy indexer won't work with NaN 
# so we have to set a flag to False
mask = df.tweet_text.str.contains(ph_raw, na=False)

In [310]:
df[mask][:5]

,twitter_screenname,date_tweet_sent,tweet_text,times_retweeted,times_favorited,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
2805,peterkistner,7/20/2016 12:26,RT @SPIEGELONLINE: In Dänemark hat eine 49-Jäh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36987,puredavie,7/30/2016 15:13,#IHatePokemonGoBecause my husband wants to cal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49204,marianeuberg1,7/20/2016 12:18,RT @SPIEGELONLINE: In Dänemark hat eine 49-Jäh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49362,puredavie,7/30/2016 16:33,RT @DanaGeezus: #IHatePokemonGoBecause now I ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50021,mr_clampin,8/16/2016 9:39,RT @SuperGeekGirls: One of our stunning @super...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pokemon Go fans

So let's take someone who has tweeted about Pokemon at random.

In [352]:

subject = df[ df.twitter_screenname == 'traceyhappymom']
stuff = subject.tweet_text.values
stuff = stuff.reshape((-1,))

In [345]:
set(hashtag_regex.findall(' '.join(stuff)))
print

In [358]:
tp = re.compile('trump',re.IGNORECASE)
pp = re.compile('putin',re.IGNORECASE)
hp = re.compile('hillary',re.IGNORECASE)
sp = re.compile('sanders',re.IGNORECASE)
cp = pp
mask = subject.tweet_text.str.contains(cp,na=False)
print '\n'.join(subject[mask].tweet_text.values.reshape((-1,)) )

RT @JohnFPLane: Just spoke to Putin. Fantastic, great guy. Knows a great deal when he sees it. Sold him the White House! #probabletrumpstwe…
RT @DoompaAlioompa: #ProbableTrumpsTweets
I'll be Putin on the Ritz if you don't text back
RT @Born_To_DYE: #TrumpsFavoriteHeadline Putin admits: 'Trump is my hero.'
RT @WhiteCollarCMDY: #ProbableTrumpsTweets
Invites go out to Putin, Jong-un, Khomeini, Duterte and Mugabe--poker at my house, winner gets t…


# Rough Analysis

This is just screwing around to see what is in the file.

### Todo

I should really feed this through an **NLP** pipeline.

In [311]:
AA = df.twitter_screenname.values
BB = df.tweet_text.values

In [58]:
names = set( AA.reshape((-1,)))

In [63]:
txts =  list(BB.reshape((-1,)))

In [72]:
txts = [x.replace('\n',' ') + '\n' for x in txts if type(x) == str]

so here there are a load of texts about Pokemon.

- What does this prove? 
- Why didn't they clean the data?
- Do those pesky Russians spies want to prove they are real people?

In [92]:
pokemon = [x for x in txts if 'pokemon' in x.lower()]
len(pokemon)

181

In [257]:
import re
df.filter(items=['Tweet text'], regex=r'#[Pp]okemon')

TypeError: Keyword arguments `items`, `like`, or `regex` are mutually exclusive

# Hashtag analysis

I'm not going to do anything complicated but here we go

In [111]:
data = file('amalgam_txts.txt','r').read()

In [322]:
tags = hashtag_regex.findall(data)
tags = list(set(tags))
tags.sort()

In [247]:
clinton = [x for x in tags if 'clinton' in x.lower()]
clinton = list(set(clinton))
clinton.sort()

In [249]:
#associated_words(clinton,['clintons','hilary'])

In [158]:
bill = [x for x in clinton if 'bill' in x.lower()]
hil = [x for x in clinton if 'hillary' in x.lower() ]

we have to write a filter to clean things up 
using a list of **stop words**

In [234]:
def associated_words(tags, omit=['']):
    stop_words = omit + ['bill','hillary','clinton','2016','16','_'] 
    old = ' '.join(tags)
    stuff = old.lower()
    for x in stop_words:
        stuff = stuff.replace(x,'')
    return set(stuff.split())

In [233]:
associated_words(hil)

{'arrest',
 'corrupt',
 'criminal',
 'crooked',
 'forpresident',
 'indict',
 'rotten',
 'scripted',
 'searchterms',
 'theliesof'}

In [228]:
obama = [x for x in tags if 'obama' in x.lower()]

In [235]:
associated_words(obama, omit=['obama','barack'])

{'44',
 '4prison',
 '7wars',
 '99daysof',
 'admin',
 'administration',
 'arrest',
 'arrestnow',
 'b',
 'bibivs',
 'birdlaunchby',
 'birthday',
 'birther',
 'bomb',
 'bye',
 'byebye',
 'ca',
 'care',
 'carefail',
 'careinthreewords',
 'coalition',
 'corrupt',
 'countdown',
 'criesabout',
 'crooked',
 'dance',
 'day',
 'defeat',
 'fail',
 'failed',
 'failedhopeandchange',
 'failedus',
 'family',
 'farew',
 'farewell',
 'farewelladdress',
 'farewells',
 'finaldays',
 'foreignpolicy',
 'forprison',
 'fuck',
 'g',
 'gate',
 'goodbye',
 'goodjob',
 'goodriddance',
 'growapair',
 'happy',
 'happybirthday',
 'happynday',
 'hometown',
 'husseininsane',
 'ilove',
 'imgoingtomissbecause',
 'impeach',
 'impeachnow',
 'imprison',
 'indict',
 'insult',
 'isafraud',
 'iscancer',
 'isisstra',
 'isisstrategies',
 'istheproblem',
 'istheworstpresidentever',
 'jail',
 'king',
 'knew',
 'legacy',
 'legacyoffailures',
 'lies',
 'malia',
 'michelle',
 'michellemade',
 'n',
 'nation',
 'ncare',
 'nextjob',
 

In [241]:
america = [x for x in tags if 'america' in x.lower()]
associated_words(america, omit=['american','america'])

{'1st',
 '4',
 '4alls',
 'actfor',
 'african',
 'africanhomelandstates',
 'africans',
 'allrejects',
 'anti',
 'antirhetoric',
 'antitrumpis',
 'bendover',
 'black',
 'blackpbs',
 'blacks',
 'bornagain',
 'callbank',
 'cgi',
 'contractwith',
 'dad',
 'death2',
 'deathto',
 'decides',
 'democratsdestructionof',
 'deservesbetter',
 'disgraceto',
 'divided',
 'eagle',
 'eagleday',
 'energyplan',
 'f',
 'failedslogans',
 'fake',
 'falls',
 'feminine4trump',
 'fi',
 'fir',
 'firs',
 'first',
 'flag',
 'for',
 'fuckingsinrealtime',
 'fundamentallychanged',
 'getitright',
 'godbless',
 'gods',
 'gophatess',
 'gopunfit2leadorgoverns',
 'greats',
 'grindingdown',
 'guns',
 'gunsin',
 'hates',
 'hatess',
 'heartassociation',
 'hire',
 'historyisblack',
 'honest',
 'horrorstory',
 'hostage',
 'howtosurvivetrumps',
 'idol',
 'ilove',
 'include',
 'inspiring',
 'isback',
 'isdoomed',
 'ismnotglobalism',
 'keepbeautiful',
 'keepsafeagain',
 'lagalaxyvs',
 'lefthates',
 'letsmakegreat',
 'letsmakegre